# Models

## Functional API

Like most models in `kgcnn.literature` the models can be set up with the `keras` functional API. Here an example for a simple message passing GNN. The layers are taken from `kgcnn.layers` . See documentation of layers for further details.

## Subclassing Model

A model can be constructed by subclassing from `keras.models.Model` where the call method must be implemented. 

Also layers can be further subclassed to create a GNN, for example of the message passing base layer. Where only `message_function` and `update_nodes` must be implemented.

## Loading options

There are many options to load data to a keras model, which depend on the size and location of the data to pass to the model. There may differences in speed and utility depending on the loading method. For more examples, please find https://github.com/aimat-lab/gcnn_keras/blob/master/notebooks/tutorial_model_loading_options.ipynb .

> **NOTE**: You can find this page as jupyter notebook in https://github.com/aimat-lab/gcnn_keras/tree/master/docs/source